In [24]:
import requests
import json
import pandas as pd
import pyodbc
from urllib.parse import quote_plus
from sqlalchemy import create_engine
from sqlalchemy.orm.session import sessionmaker
import numpy as np
import re
import collections
from collections import Counter

conn_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=192.168.189.71;DATABASE=Dokuchaev;UID=sa;PWD=As12345^'
quoted = quote_plus(conn_string)
new_conn_string = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(new_conn_string)

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
%%time
payments_rd = pd.read_sql('SELECT * FROM dbo.payments', engine)
payments_rd = payments_rd.loc[payments_rd['Проведено банком']=='Да']

Wall time: 31.2 s


In [3]:
payments_rd[payments_rd['Поступление / списание'].str.contains('Списание')] =\
    payments_rd[payments_rd['Поступление / списание'].str.contains('Списание')].drop_duplicates(
    subset=['Тип платежного документа', 'Дата входящего документа',
            'Номер входящего документа', 'Сумма документа'])
payments_rd = payments_rd.dropna(subset=['Сумма документа'])

In [4]:
payments_rd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13081 entries, 0 to 13130
Data columns (total 77 columns):
False                                          13081 non-null float64
Поступление / списание                         13081 non-null object
Организация                                    13081 non-null object
Банковский счет                                13081 non-null object
Тип платежного документа                       13081 non-null object
Операция                                       13081 non-null object
Плательщик                                     7861 non-null object
Банковский счет контрагента                    5504 non-null object
Банковский счет отправитель                    12 non-null object
Касса отправитель                              319 non-null object
Дата входящего документа                       13081 non-null datetime64[ns]
Номер входящего документа                      13081 non-null object
Назначение платежа                             13073 non-null

In [5]:
payments = pd.DataFrame()
payments['type'] = payments_rd['Поступление / списание'].apply(lambda x:
          'Приход' if 'Поступление' in str(x) \
                else 'Расход' if 'Списание' in str(x) \
                else '')
pattern = 'ДС \w{,5}?-\w{,8}? от \d\d.\d\d.\d\d\d\d?'
payments['document_number'] = payments_rd['Поступление / списание'].apply(lambda x:
                                               str(re.findall(pattern, x))[5:16])
payments['payment_date'] = payments_rd['Дата проведения банком']
payments['cf_item'] = payments_rd[['Приход денежных средств', 'Расход денежных средств']].\
    apply(lambda x: x[0] if x[0] != None else x[1] if x[1] != None else None, axis=1)

In [6]:
payments['operation'] = payments_rd['Операция']
payments['department'] = payments_rd['Подразделение']
payments['partner'] = payments_rd['Партнер']
payments['purpose'] = payments_rd['Назначение платежа']
payments['amount'] = payments_rd['Сумма документа'].fillna(0)
payments.amount = payments[['type','amount']].apply(lambda x: x[1] if x[0] == 'Приход' else -x[1], axis=1)
payments['currency'] = payments_rd['Валюта'] 
payments['acount'] = payments_rd['Банковский счет'].apply(lambda x:
                    str(re.findall('\d{20}?', x))[2:-2])
payments['bank'] = payments_rd['Банковский счет'].apply(lambda x:
                    str(x)[22:] if re.search('\d{20}', x) !=None else None)
payments['reciever']=  payments_rd['Получатель']
payments.fillna(value='', inplace=True)

In [7]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13081 entries, 0 to 13130
Data columns (total 13 columns):
type               13081 non-null object
document_number    13081 non-null object
payment_date       13081 non-null datetime64[ns]
cf_item            13081 non-null object
operation          13081 non-null object
department         13081 non-null object
partner            13081 non-null object
purpose            13081 non-null object
amount             13081 non-null float64
currency           13081 non-null object
acount             13081 non-null object
bank               13081 non-null object
reciever           13081 non-null object
dtypes: datetime64[ns](1), float64(1), object(11)
memory usage: 1.4+ MB


In [23]:
payments[(payments.currency == 'RUB') & (payments.amount < 0) & (payments.payment_date.dt.month <= 6)]\
    [['acount', 'amount']].groupby(['acount']).sum()

,amount
acount,
,-3984545.66
40702810001600004073,-15693881.46
40702810200000038305,-277807344.83
40702810438000032207,-1141089546.79
40702810603000020391,-31993230.91


In [9]:
for i in range(1,30):
    a = payments[(payments.payment_date.dt.month == 2)
         & (payments.payment_date.dt.day == i)
         & (payments.currency =='RUB')
         & (payments.acount =='40702810438000032207')
         & (payments.type =='Расход')
        ].amount.sum()
    print(i,a)

1 -1224757.84
2 -1727268.81
3 -10753045.120000001
4 -2296274.34
5 -52272449.69
6 -4335952.289999999
7 -11655290.62
8 -1249233.62
9 -590164.59
10 -19924586.17
11 -5189980.85
12 -43746848.53
13 -24380988.83
14 -24658531.91
15 -1556420.08
16 -826692.92
17 -7678577.5
18 -23155681.950000003
19 -9288601.129999999
20 -17749718.86
21 -4960399.16
22 -1547314.3
23 -628313.66
24 -856515.28
25 -15904359.39
26 -10142302.600000001
27 -10773350.379999999
28 -7057507.199999999
29 -2163053.7800000003


In [49]:
#set(
payments[(payments.payment_date.dt.month <= 6)
#         & (payments.payment_date.dt.day == 14)
#         & (payments.currency =='RUB')
#         & (payments.type =='Расход')
         & (payments.partner.isin(foreign_partners))
        ].sort_values('payment_date').to_excel('G:/Мой диск/04 FS/2020 M 07/Global CF/items.xlsx',
                                             index=False)#.partner)
    

In [36]:
foreign_partners = ['Blazej Blazej','LEE LIANG HSIEN',
                    'MEMENTO IMPORTACAO E EXPORTACAO E SERVICOS',
                    'NATAL E MANSSUR SOCIEDADE DE ADVOGADOS',
                    'COFIX POLSKA SP. Z.O.O.'
                   ]

In [12]:
payments[(payments.payment_date.dt.month == 2)
         & (payments.payment_date.dt.day == 14)
         & (payments.currency =='RUB')
         & (payments.acount =='40702810438000032207')
         & (payments.type =='Расход')
        ]

,type,document_number,payment_date,cf_item,operation,department,partner,purpose,amount,currency,acount,bank,reciever
2620,Расход,00УП-000469,2020-02-14,Прочие,Прочий расход ДС,Основное подразделение (УКР),,Комиссия внутри Сбербанка за ПП/ПТ через ДБО с...,-8.000000000000E+00,RUB,40702810438000032207,ПАО СБЕРБАНК,ПАО СБЕРБАНК (ком.)
2621,Расход,00УП-000470,2020-02-14,,Инкассация ДС в банк,,,НДС с платы (неустойки) за несвоевременный отк...,-69.170000000000E+00,RUB,40702810438000032207,ПАО СБЕРБАНК,
2622,Расход,00УП-000471,2020-02-14,Прочие,Прочий расход ДС,Основное подразделение (УКР),,Комиссия в другие банки (кредитные организации...,-105.000000000000E+00,RUB,40702810438000032207,ПАО СБЕРБАНК,ПАО СБЕРБАНК (ком.)
2623,Расход,00УП-000472,2020-02-14,,Инкассация ДС в банк,,,Плата (неустойка) за несвоевременный отказ от ...,-345.830000000000E+00,RUB,40702810438000032207,ПАО СБЕРБАНК,
2624,Расход,00УП-П01242,2020-02-14,Прочие,Перечисление налогов и взносов,Основное подразделение (УКР),,Государственная пошлина за выдачу приглашения ...,-800.000000000000E+00,RUB,40702810438000032207,ПАО СБЕРБАНК,"УФК по г. Москве (ГУ МВД России по г. Москве, ..."
2625,Расход,00УП-П01243,2020-02-14,Зарплата,Выплата по ведомости по зарплатному проекту,Отдел бухгалтерского учета и отчетности,,Материальная помощь по реестру №71 от 14.02.20...,-4.000000000000E+03,RUB,40702810438000032207,ПАО СБЕРБАНК,СБЕРБАНК ПАО
2626,Расход,00УП-000473,2020-02-14,Прочие,Прочий расход ДС,Основное подразделение (УКР),,Комиссия за перечисление средств со сч. ЮЛ на ...,-4.900000000000E+03,RUB,40702810438000032207,ПАО СБЕРБАНК,ПАО СБЕРБАНК (ком.)
2627,Расход,00УП-П01244,2020-02-14,ИТ Услуги,Оплата поставщику,Основное подразделение (УКР),РАЙС ООО,Оплата сч. №5 от 11.02.2020 Выполнение работ п...,-125.400000000000E+03,RUB,40702810438000032207,ПАО СБЕРБАНК,РАЙС ООО
2628,Расход,00УП-П01245,2020-02-14,Займ,Выдача займа сотруднику,Основное подразделение (УКР),,Для зачисления на счет Раджабовой Карины Выдач...,-140.000000000000E+03,RUB,40702810438000032207,ПАО СБЕРБАНК,
2629,Расход,00УП-П01246,2020-02-14,Аренда франчайзи,Оплата поставщику,Невский 48_ТЦ_СПБ,НЕВСКИЙ ПАССАЖ ООО,Фиксированная арендная плата за период с 01.03...,-356.689130000000E+03,RUB,40702810438000032207,ПАО СБЕРБАНК,НЕВСКИЙ ПАССАЖ ООО


In [13]:
payments[payments.document_number=='00УП-001256']

,type,document_number,payment_date,cf_item,operation,department,partner,purpose,amount,currency,acount,bank,reciever
69,Расход,00УП-001256,2020-06-15,Прочие,Прочий расход ДС,Основное подразделение (УКР),,НДС за инкассацию денежной наличности по дог. ...,-140.000000000000E+00,RUB,40702810438000032207,ПАО СБЕРБАНК,ПАО СБЕРБАНК (ком.)
8916,Приход,00УП-001256,2020-01-24,"Инкассация, эквайринг",Инкассация ДС из кассы на расчетный счет,,,Инкасс. наличность по дог. N ЕД/1771/0029/У от...,79.950000000000E+03,RUB,40702810438000032207,ПАО СБЕРБАНК,


In [14]:
Counter(payments.acount).most_common(50)

[('40702810438000032207', 8974),
 ('40702810200000038305', 3168),
 ('40702810603000020391', 815),
 ('40702810001600004073', 75),
 ('40702840938000009234', 22),
 ('40702978938000007836', 15),
 ('', 8),
 ('40702840238001009234', 4)]

In [15]:
payments_rd.loc[payments_rd['Партнер']=='Сименс Финанс ООО'].loc[1420]

False                                                      9.490000000000E+03
Поступление / списание      Списание безналичных ДС 00УП-П03100 от 01.04.2...
Организация                                             УРБАН КОФИКС РАША ООО
Банковский счет                                 , УРБАН КОФИКС РАША ООО (руб)
Тип платежного документа                                  Платежное поручение
                                                  ...                        
КПП Плательщика                                                          None
Текст плательщика                                                        None
Расход денежных средств                               Расходы на оборудование
Очередность платежа                                        5.000000000000E+00
Счёт                                                     355.000000000000E+00
Name: 1420, Length: 77, dtype: object

In [16]:
set(payments.sort_values('payment_date').acount)

{'',
 '40702810001600004073',
 '40702810200000038305',
 '40702810438000032207',
 '40702810603000020391',
 '40702840238001009234',
 '40702840938000009234',
 '40702978938000007836'}

In [17]:
list(set(payments.department.values))

['',
 'Бутово Молл_ТЦ_МСК',
 'Парнас_ТЦ_СПБ',
 'Галерея Аэропорт_ТЦ_МСК',
 'Финансовый департамент',
 'Калужский_Новый_ТЦ_МСК',
 'НЕВСКИЙ, 134Б',
 'Планерная',
 'Отдел качества',
 'Операционный отдел (Санкт-Петербург)',
 'Отдел обучения персонала',
 'Золотой Вавилон Ростокино_ТЦ МСК',
 'Водный_ТЦ_МСК',
 'Санкт-Петербург (офис)',
 'Операционный отдел',
 'Бауманский',
 'Город Рязанка_ТЦ_МСК',
 'Город Лефортово_ТЦ_МСК',
 'Фудтрак 1_ФТ_МСК',
 'Б.Тульская_11_ТДК_МСК',
 'Отдел коммуникаций',
 'Корпоративные кофейни СПБ',
 'Филион_ТЦ_МСК',
 'Радуга',
 'Отдел франчайзинга',
 'Измайловское ш',
 'Основное подразделение СПБ',
 'Отдел разработки продуктов',
 'Румянцево_БЦ_МСК',
 'Хорошо_ТЦ_МСК',
 'Отдел закупок',
 'Ленинградский проспект',
 'отдел по обработке и анализу данных',
 'Ереван Плаза_1_ТЦ_МСК',
 'Метрополис 3_ТЦ_МСК',
 'ЛИГОВСКИЙ ПРОСПЕКТ, Д. 48',
 'Каменноостровский, 40А',
 'Ореховый бульвар 22_ТЦ_МСК',
 'Красный Кит_ТЦ_МО',
 'Департамент по производству',
 'Пресненская 12_ТЦ_МСК',
 'От

In [18]:
list(set(payments.cf_item.values))

['',
 'Роялти',
 'Зарплата',
 'Расходы на оборудование',
 'Расходы на прочие услуги',
 'Овердрафт',
 'Аренда франчайзи',
 'Прочие услуги офиса',
 'Инкассация, эквайринг',
 'Оказание услуг по приготовлению напитков, контроль качества',
 'Расходы на маркетинг',
 'Займ',
 'Налог на прибыль',
 'ИТ Услуги',
 'Аренда собственная',
 'Первоначальный взнос',
 'Страхование',
 'Товары',
 'Взносы',
 'Овердрафт проценты',
 'Мероприятия',
 'Прочие',
 'Перевод на другой счет ор',
 'Маркетинговый взнос',
 'Займы',
 'Услуги, консалтинг, тренинги',
 'Аудит',
 'Прочее',
 'Налоги',
 'НДС',
 'Обеспечительный платеж']

In [50]:
%%time

#соединение с БД
conn_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=192.168.189.71;DATABASE=Dokuchaev;UID=sa;PWD=As12345^'
quoted = quote_plus(conn_string)
new_conn_string = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(new_conn_string)

Session = sessionmaker(bind=engine)
session = Session()

payments.to_sql('payments_new', engine, schema = 'dbo', if_exists='replace', chunksize=None, index_label=False)

Wall time: 15min 4s
